In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 1: SETUP, IMPORTS, AND CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

# Library Imports
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════
# CONSTANTS AND DIRECTORIES
# ═══════════════════════════════════════════════════════════════════════════

DATA_ROOT = '/content/drive/MyDrive/crema_d/AudioWAV/'
OUTPUT_DIR = './experiments_log/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Seed for Reproducibility
SEED = 42
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {DEVICE}")

# Emotion Label Mapping
LABEL_MAP = {'A': 0, 'D': 1, 'F': 2, 'H': 3, 'N': 4, 'S': 5}
NUM_CLASSES = len(LABEL_MAP)
EMOTION_LABELS = list(LABEL_MAP.keys())

# Audio Feature Settings
TARGET_SR = 22050
MAX_LENGTH = 3.5
N_FFT = 1024
HOP_LENGTH = 512
N_MELS = 128
N_MFCC = 40

# Global Variable for Normalization Stats
FEATURE_STATS = {}

# ═══════════════════════════════════════════════════════════════════════════
# CELL 2: DATA LOADING AND PREPROCESSING
# ═══════════════════════════════════════════════════════════════════════════

def load_audio(file_path):
    """Load audio at target sample rate and pad/truncate to fixed length."""
    audio, sr = librosa.load(file_path, sr=TARGET_SR)
    max_samples = int(MAX_LENGTH * sr)
    if len(audio) > max_samples:
        audio = audio[:max_samples]
    elif len(audio) < max_samples:
        padding = np.zeros(max_samples - len(audio))
        audio = np.concatenate((audio, padding))
    return audio

# CREMA-D Emotion Code Mapper
EMOTION_CODE_MAPPER = {
    'ANG': 'A', 'DIS': 'D', 'FEA': 'F', 'HAP': 'H', 'NEU': 'N', 'SAD': 'S'
}
VALID_EMOTION_CODES_3CHAR = set(EMOTION_CODE_MAPPER.keys())

def create_cremad_dataframe(data_root):
    """Parse CREMA-D filenames and create DataFrame."""
    file_paths = []
    actor_ids = []
    emotions = []

    for filename in os.listdir(data_root):
        if filename.endswith(".wav"):
            base_filename = filename[:-4]
            parts = base_filename.split('_')

            if len(parts) < 4:
                continue

            emotion_code_3char = parts[2].upper()

            if emotion_code_3char not in VALID_EMOTION_CODES_3CHAR:
                continue

            emotion_code_1char = EMOTION_CODE_MAPPER[emotion_code_3char]
            actor_id = parts[0]

            file_paths.append(os.path.join(data_root, filename))
            actor_ids.append(actor_id)
            emotions.append(emotion_code_1char)

    df = pd.DataFrame({
        'path': file_paths,
        'actor_id': actor_ids,
        'emotion': emotions
    })

    print(f"📊 Total files processed: {len(df)}")
    return df

# Load and Split Data (Speaker-Independent)
df_full = create_cremad_dataframe(DATA_ROOT)
actor_list = df_full['actor_id'].unique()
train_actors, test_actors = train_test_split(
    actor_list, test_size=0.2, random_state=SEED
)

train_df = df_full[df_full['actor_id'].isin(train_actors)]
test_df = df_full[df_full['actor_id'].isin(test_actors)]

# Calculate Class Weights
emotion_counts = train_df['emotion'].value_counts()
total_samples = emotion_counts.sum()
class_weights = total_samples / (NUM_CLASSES * emotion_counts)
sorted_weights = [class_weights[e] for e in EMOTION_LABELS]
WEIGHTS_TENSOR = torch.tensor(sorted_weights, dtype=torch.float32).to(DEVICE)

print(f"👥 Training Speakers: {len(train_actors)}, Test Speakers: {len(test_actors)}")
print(f"⚖️  Class Weights: {WEIGHTS_TENSOR.cpu().numpy()}")

# ═══════════════════════════════════════════════════════════════════════════
# CELL 3: PREPROCESSING PIPELINES
# ═══════════════════════════════════════════════════════════════════════════

def preproc_A(audio):
    """Pipeline A: Baseline - Simple Normalization."""
    return audio

def preproc_B(audio, sr=TARGET_SR):
    """Pipeline B: Data Augmentation (Pitch Shift + Noise)."""
    pitch_shifted = librosa.effects.pitch_shift(
        audio, sr=sr, n_steps=random.uniform(-4, 4)
    )
    noise_amp = 0.005 * np.random.uniform() * np.amax(pitch_shifted)
    noisy_audio = pitch_shifted + noise_amp * np.random.normal(size=pitch_shifted.shape[0])
    return noisy_audio

def preproc_C(audio):
    """Pipeline C: RMS Normalization."""
    rms = np.sqrt(np.mean(audio**2))
    normalized_audio = audio / (rms + 1e-6)
    return normalized_audio

PREPROC_PIPELINES = {
    'A_Baseline': preproc_A,
    'B_Augmentation': preproc_B,
    'C_RMS_Norm': preproc_C
}

# ═══════════════════════════════════════════════════════════════════════════
# CELL 4: FEATURE EXTRACTION PIPELINES
# ═══════════════════════════════════════════════════════════════════════════

def features_A(audio, sr=TARGET_SR):
    """Features A: MFCC + Delta + Delta-Delta."""
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    combined_features = np.concatenate((mfcc, mfcc_delta, mfcc_delta2), axis=0)
    return combined_features.T

def features_B(audio, sr=TARGET_SR):
    """Features B: Log-Mel Spectrogram + Delta + Delta-Delta (3-Channel)."""
    mel_spectrogram = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS
    )
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    delta = librosa.feature.delta(log_mel_spectrogram)
    delta2 = librosa.feature.delta(log_mel_spectrogram, order=2)
    combined_features = np.stack([log_mel_spectrogram, delta, delta2], axis=0)
    return combined_features

def features_C(audio, sr=TARGET_SR):
    """Features C: Hybrid MFCC + Spectral Features."""
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13, n_fft=N_FFT, hop_length=HOP_LENGTH)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH)
    centroid = librosa.feature.spectral_centroid(y=audio, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH)
    bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH)
    combined_features = np.concatenate((mfcc, chroma, centroid, bandwidth), axis=0)
    return combined_features.T

FEATURE_PIPELINES = {
    'A_MFCC_Delta': features_A,
    'B_LogMelSpec_3Ch': features_B,
    'C_Hybrid_1D': features_C
}

# ═══════════════════════════════════════════════════════════════════════════
# CELL 5: SPECAUGMENT CLASS (NEW!)
# ═══════════════════════════════════════════════════════════════════════════

class SpecAugment:
    """
    SpecAugment: Time and Frequency Masking for Spectrograms.
    Reference: Park et al., 2019 - SpecAugment
    """
    def __init__(self, freq_mask_param=15, time_mask_param=20,
                 num_freq_masks=1, num_time_masks=2):
        self.freq_mask_param = freq_mask_param
        self.time_mask_param = time_mask_param
        self.num_freq_masks = num_freq_masks
        self.num_time_masks = num_time_masks

    def __call__(self, spec):
        """Apply SpecAugment to spectrogram tensor."""
        spec = spec.clone()

        if len(spec.shape) == 2:
            spec = spec.unsqueeze(0)
            squeeze_output = True
        else:
            squeeze_output = False

        channels, n_freqs, n_frames = spec.shape

        # Frequency Masking
        for _ in range(self.num_freq_masks):
            f = random.randint(0, self.freq_mask_param)
            f0 = random.randint(0, max(0, n_freqs - f))
            spec[:, f0:f0+f, :] = 0

        # Time Masking
        for _ in range(self.num_time_masks):
            t = random.randint(0, min(self.time_mask_param, n_frames))
            t0 = random.randint(0, max(0, n_frames - t))
            spec[:, :, t0:t0+t] = 0

        if squeeze_output:
            spec = spec.squeeze(0)

        return spec

# ═══════════════════════════════════════════════════════════════════════════
# CELL 6: DATASET CLASS WITH SPECAUGMENT
# ═══════════════════════════════════════════════════════════════════════════

class SERDataset(Dataset):
    """Speech Emotion Recognition Dataset with Optional SpecAugment."""
    def __init__(self, df, preproc_name, feature_name, stats=None, augment=False):
        self.df = df
        self.preproc = PREPROC_PIPELINES[preproc_name]
        self.feature_extractor = FEATURE_PIPELINES[feature_name]
        self.labels = [LABEL_MAP[l] for l in df['emotion']]
        self.stats = stats
        self.augment = augment

        if self.augment:
            self.spec_augment = SpecAugment(
                freq_mask_param=15,
                time_mask_param=20,
                num_freq_masks=1,
                num_time_masks=2
            )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df.iloc[idx]['path']
        label = self.labels[idx]

        audio = load_audio(file_path)
        audio = self.preproc(audio)
        features = self.feature_extractor(audio)

        features_tensor = torch.tensor(features, dtype=torch.float32)

        # Apply SpecAugment BEFORE normalization (only for 2D/3D features)
        if self.augment and len(features_tensor.shape) >= 2:
            features_tensor = self.spec_augment(features_tensor)

        # Normalize
        if self.stats:
            mean = self.stats['mean']
            std = self.stats['std']
            features_tensor = (features_tensor - mean) / (std + 1e-6)

        return features_tensor, torch.tensor(label, dtype=torch.long)

# ═══════════════════════════════════════════════════════════════════════════
# CELL 7: MODEL ARCHITECTURES
# ═══════════════════════════════════════════════════════════════════════════

ACTIVATIONS = {
    'relu': nn.ReLU(),
    'leakyrelu': nn.LeakyReLU(),
    'gelu': nn.GELU()
}

def calculate_conv_output_dim(input_tensor_shape, model):
    """Calculate flattened dimension after conv layers."""
    dummy_input = torch.zeros(1, *input_tensor_shape).to(DEVICE)
    with torch.no_grad():
        output = model.features(dummy_input)
        flattened_size = output.numel()
        return flattened_size // output.shape[0]

# ─────────────────────────────────────────────────────────────────────────────
# MODEL 1: CNN
# ─────────────────────────────────────────────────────────────────────────────

class Model_CNN_A(nn.Module):
    def __init__(self, input_shape, num_classes, activation='relu', dropout_rate=0.5):
        super().__init__()
        act = ACTIVATIONS[activation]

        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32), act, nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(dropout_rate),
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64), act, nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(dropout_rate)
        )

        flattened_size = calculate_conv_output_dim(input_shape, self)

        self.classifier = nn.Sequential(
            nn.Linear(flattened_size, 256), act,
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ─────────────────────────────────────────────────────────────────────────────
# MODEL 2: LSTM
# ─────────────────────────────────────────────────────────────────────────────

class Model_LSTM_A(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size=128, num_layers=2,
                 activation='relu', dropout_rate=0.5):
        super().__init__()
        act = ACTIVATIONS[activation]
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,
                            bidirectional=True, dropout=dropout_rate if num_layers > 1 else 0)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 64), act,
            nn.Dropout(dropout_rate),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        final_state = lstm_out[:, -1, :]
        out = self.fc(final_state)
        return out

# ─────────────────────────────────────────────────────────────────────────────
# MODEL 3: CRNN (Standard)
# ─────────────────────────────────────────────────────────────────────────────

class Model_CRNN_A(nn.Module):
    def __init__(self, input_shape, num_classes, hidden_size=64, num_layers=1,
                 activation='relu', dropout_rate=0.5):
        super().__init__()
        act = ACTIVATIONS[activation]

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 16, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(16), act, nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(dropout_rate)
        )
        self.features = self.conv

        feature_size_after_conv = 16 * input_shape[1]

        self.rnn = nn.LSTM(feature_size_after_conv, hidden_size, num_layers, batch_first=True,
                           bidirectional=True, dropout=dropout_rate if num_layers > 1 else 0)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.permute(0, 3, 1, 2)
        x = x.reshape(x.size(0), x.size(1), -1)
        rnn_out, _ = self.rnn(x)
        final_state = rnn_out[:, -1, :]
        out = self.classifier(final_state)
        return out

# ─────────────────────────────────────────────────────────────────────────────
# MODEL 4: CRNN WITH ATTENTION (NEW!)
# ─────────────────────────────────────────────────────────────────────────────

class Model_CRNN_Attention(nn.Module):
    """CRNN with Self-Attention Mechanism."""
    def __init__(self, input_shape, num_classes, hidden_size=64, num_layers=1,
                 activation='gelu', dropout_rate=0.5):
        super().__init__()
        act = ACTIVATIONS[activation]

        # Convolutional Feature Extractor
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32), act, nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(dropout_rate),
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64), act, nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(dropout_rate)
        )
        self.features = self.conv

        feature_size_after_conv = 64 * input_shape[1]

        # Bidirectional LSTM
        self.rnn = nn.LSTM(
            feature_size_after_conv,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        # Self-Attention Mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )

        # Classification Head
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size * 2, 128),
            act,
            nn.Dropout(dropout_rate),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # Conv layers
        x = self.conv(x)

        # Reshape for RNN
        x = x.permute(0, 3, 1, 2)
        x = x.reshape(x.size(0), x.size(1), -1)

        # RNN
        rnn_out, _ = self.rnn(x)

        # Attention mechanism
        attn_weights = self.attention(rnn_out)
        attn_weights = torch.softmax(attn_weights, dim=1)

        # Weighted sum (context vector)
        context = torch.sum(rnn_out * attn_weights, dim=1)

        # Classification
        out = self.classifier(context)
        return out

# ═══════════════════════════════════════════════════════════════════════════
# CELL 8: EVALUATION FUNCTION
# ═══════════════════════════════════════════════════════════════════════════

def evaluate_model(model, data_loader, stage, return_full=False):
    """Evaluate model and return metrics."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for features, labels in data_loader:
            if len(features.shape) > 2 and features.shape[1] == 1:
                features = features.squeeze(1)
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    if return_full:
        cm = confusion_matrix(all_labels, all_preds)
        report = classification_report(all_labels, all_preds, target_names=EMOTION_LABELS, zero_division=0)
        return accuracy, f1, cm, report
    return accuracy, f1

# ═══════════════════════════════════════════════════════════════════════════
# CELL 9: TRAINING FUNCTION WITH SCHEDULER (FIXED!)
# ═══════════════════════════════════════════════════════════════════════════

def train_and_evaluate(config, train_df, test_df, weights_tensor):
    """Train and evaluate a single experiment configuration with LR Scheduler."""

    preproc_name = config['preproc']
    feature_name = config['features']
    batch_size = config['batch_size']
    epochs = config['epochs']

    print(f"\n{'='*80}")
    print(f"🚀 EXPERIMENT: {config['name']}")
    print(f"{'='*80}")

    # ─────────────────────────────────────────────────────────────────────────
    # Step 1: Calculate Normalization Statistics
    # ─────────────────────────────────────────────────────────────────────────

    temp_dataset = SERDataset(train_df, preproc_name, feature_name, stats=None, augment=False)
    temp_loader = DataLoader(temp_dataset, batch_size=len(temp_dataset), shuffle=False)
    features_all, _ = next(iter(temp_loader))

    if len(features_all.shape) > 2 and features_all.shape[1] == 1:
        features_all = features_all.squeeze(1)

    mean = features_all.mean(dim=0, keepdim=True).to('cpu')
    std = features_all.std(dim=0, keepdim=True).to('cpu')

    global FEATURE_STATS
    FEATURE_STATS[config['name']] = {'mean': mean, 'std': std}

    # ─────────────────────────────────────────────────────────────────────────
    # Step 2: Create Data Loaders with SpecAugment
    # ─────────────────────────────────────────────────────────────────────────

    train_dataset = SERDataset(
        train_df, preproc_name, feature_name,
        stats=FEATURE_STATS[config['name']],
        augment=True  # Enable SpecAugment for training
    )
    test_dataset = SERDataset(
        test_df, preproc_name, feature_name,
        stats=FEATURE_STATS[config['name']],
        augment=False  # No augmentation for testing
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # ─────────────────────────────────────────────────────────────────────────
    # Step 3: Model Instantiation
    # ─────────────────────────────────────────────────────────────────────────

    sample_features, _ = next(iter(train_loader))
    input_shape = sample_features.shape

    if len(input_shape) > 2 and input_shape[1] == 1:
        sample_features = sample_features.squeeze(1)
        input_shape = sample_features.shape

    # Skip LSTM if input is 4D
    if config['model_type'] == 'LSTM_A' and len(input_shape) == 4:
        print(f"⚠️  EXPERIMENT SKIPPED: LSTM cannot handle 4D input (Shape: {input_shape})")
        return

    model_input_shape = input_shape[1:]

    if config['model_type'] == 'LSTM_A':
        model = Model_LSTM_A(
            input_size=model_input_shape[-1],
            num_classes=NUM_CLASSES,
            activation=config['activation'],
            dropout_rate=config['dropout']
        ).to(DEVICE)
    elif config['model_type'] == 'CNN_A':
        model = Model_CNN_A(
            input_shape=model_input_shape,
            num_classes=NUM_CLASSES,
            activation=config['activation'],
            dropout_rate=config['dropout']
        ).to(DEVICE)
    elif config['model_type'] == 'CRNN_A':
        model = Model_CRNN_A(
            input_shape=model_input_shape,
            num_classes=NUM_CLASSES,
            activation=config['activation'],
            dropout_rate=config['dropout']
        ).to(DEVICE)
    elif config['model_type'] == 'CRNN_Attention':
        model = Model_CRNN_Attention(
            input_shape=model_input_shape,
            num_classes=NUM_CLASSES,
            activation=config['activation'],
            dropout_rate=config['dropout']
        ).to(DEVICE)
    else:
        raise ValueError(f"Unknown model type: {config['model_type']}")

    # ─────────────────────────────────────────────────────────────────────────
    # Step 4: Optimizer and Scheduler
    # ─────────────────────────────────────────────────────────────────────────

    if config['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    elif config['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
    elif config['optimizer'] == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])

    criterion = nn.CrossEntropyLoss(weight=weights_tensor)

    # Learning Rate Scheduler (FIXED: removed 'verbose' parameter)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )

    # ─────────────────────────────────────────────────────────────────────────
    # Step 5: Training Loop
    # ─────────────────────────────────────────────────────────────────────────

    best_val_accuracy = 0.0
    best_val_f1 = 0.0
    prev_lr = config['learning_rate']

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            if len(features.shape) > 2 and features.shape[1] == 1:
                features = features.squeeze(1)

            features, labels = features.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, labels in test_loader:
                if len(features.shape) > 2 and features.shape[1] == 1:
                    features = features.squeeze(1)

                features, labels = features.to(DEVICE), labels.to(DEVICE)
                outputs = model(features)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(test_loader)
        val_acc, val_f1 = evaluate_model(model, test_loader, 'Validation')

        # Step the scheduler
        scheduler.step(avg_val_loss)

        # Get current learning rate and check if it changed
        current_lr = optimizer.param_groups[0]['lr']
        if current_lr != prev_lr:
            print(f"📉 Learning rate reduced: {prev_lr:.6f} → {current_lr:.6f}")
            prev_lr = current_lr

        print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f} | "
              f"Val F1: {val_f1:.4f} | LR: {current_lr:.6f}")

        # Save best model
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            best_val_f1 = val_f1
            torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, f"{config['name']}_best.pth"))

    # ─────────────────────────────────────────────────────────────────────────
    # Step 6: Test Evaluation
    # ─────────────────────────────────────────────────────────────────────────

    model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, f"{config['name']}_best.pth")))
    test_acc, test_f1, cm, report = evaluate_model(model, test_loader, 'Test', return_full=True)

    print(f"\n{'─'*80}")
    print(f"📊 FINAL TEST RESULTS:")
    print(f"   Accuracy: {test_acc:.4f}")
    print(f"   F1-Score: {test_f1:.4f}")
    print(f"{'─'*80}\n")

    # ─────────────────────────────────────────────────────────────────────────
    # Step 7: Save Results
    # ─────────────────────────────────────────────────────────────────────────

    new_row = {
        'experiment_name': config['name'],
        'preprocessing_pipeline': preproc_name,
        'feature_pipeline': feature_name,
        'model_type': config['model_type'],
        'activation': config['activation'],
        'optimizer': config['optimizer'],
        'learning_rate': config['learning_rate'],
        'dropout': config['dropout'],
        'batch_size': batch_size,
        'epochs': epochs,
        'val_accuracy': best_val_accuracy,
        'val_f1': best_val_f1,
        'test_accuracy': test_acc,
        'test_f1': test_f1,
        'confusion_matrix': cm.tolist()
    }

    global RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([new_row])], ignore_index=True)
    RESULTS_DF.to_csv('experiments_results_checkpoint.csv', index=False)
    print("💾 Results saved to checkpoint file.")
# ═══════════════════════════════════════════════════════════════════════════
# CELL 10: LOAD EXISTING RESULTS OR CREATE NEW DATAFRAME
# ═══════════════════════════════════════════════════════════════════════════

csv_path = 'experiments_results_checkpoint.csv'

if os.path.exists(csv_path):
    RESULTS_DF = pd.read_csv(csv_path)
    print(f"✅ Existing results loaded: {len(RESULTS_DF)} experiments found.")
else:
    RESULTS_DF = pd.DataFrame(columns=[
        'experiment_name', 'preprocessing_pipeline', 'feature_pipeline', 'model_type',
        'activation', 'optimizer', 'learning_rate', 'dropout', 'batch_size', 'epochs',
        'val_accuracy', 'val_f1', 'test_accuracy', 'test_f1', 'confusion_matrix'
    ])
    print("🆕 New results dataframe created.")

# ═══════════════════════════════════════════════════════════════════════════
# CELL 11: RUN NEW EXPERIMENTS
# ═══════════════════════════════════════════════════════════════════════════

# ─────────────────────────────────────────────────────────────────────────────
# EXPERIMENT 9: CRNN with Attention + SpecAugment
# ─────────────────────────────────────────────────────────────────────────────

config_e09 = {
    'name': 'EXP_E09_CRNN_Attention_SpecAug',
    'preproc': 'A_Baseline',
    'features': 'B_LogMelSpec_3Ch',
    'model_type': 'CRNN_Attention',
    'activation': 'gelu',
    'optimizer': 'AdamW',
    'learning_rate': 1e-3,
    'dropout': 0.5,
    'batch_size': 32,
    'epochs': 30
}

print(f"\n{'#'*80}")
print(f"# STARTING: {config_e09['name']}")
print(f"{'#'*80}\n")

train_and_evaluate(config_e09, train_df, test_df, WEIGHTS_TENSOR)

# ─────────────────────────────────────────────────────────────────────────────
# EXPERIMENT 10: Ultimate Configuration (RMS + Attention + SpecAugment)
# ─────────────────────────────────────────────────────────────────────────────

config_e10 = {
    'name': 'EXP_E10_CRNN_Attention_RMS_SpecAug',
    'preproc': 'C_RMS_Norm',
    'features': 'B_LogMelSpec_3Ch',
    'model_type': 'CRNN_Attention',
    'activation': 'gelu',
    'optimizer': 'AdamW',
    'learning_rate': 2e-3,
    'dropout': 0.4,
    'batch_size': 24,
    'epochs': 40
}

print(f"\n{'#'*80}")
print(f"# STARTING: {config_e10['name']}")
print(f"{'#'*80}\n")

train_and_evaluate(config_e10, train_df, test_df, WEIGHTS_TENSOR)

# ═══════════════════════════════════════════════════════════════════════════
# CELL 12: FINAL VISUALIZATION AND ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════

if RESULTS_DF.empty:
    print("⚠️  No results to visualize!")
else:
    print("\n" + "="*80)
    print("📊 ALL EXPERIMENT RESULTS SUMMARY")
    print("="*80 + "\n")

    # Display summary table
    summary_cols = ['experiment_name', 'test_accuracy', 'test_f1', 'model_type']
    summary_table = RESULTS_DF[summary_cols].sort_values(by='test_f1', ascending=False)
    print(summary_table.to_string(index=False))

    # Find best model
    RESULTS_DF['test_f1'] = pd.to_numeric(RESULTS_DF['test_f1'], errors='coerce')
    best_idx = RESULTS_DF['test_f1'].idxmax()
    best_row = RESULTS_DF.loc[best_idx]

    print(f"\n{'='*80}")
    print(f"🏆 BEST MODEL: {best_row['experiment_name']}")
    print(f"   Test F1-Score: {best_row['test_f1']:.4f}")
    print(f"   Test Accuracy: {best_row['test_accuracy']:.4f}")
    print(f"   Model Type: {best_row['model_type']}")
    print(f"{'='*80}\n")

    # ─────────────────────────────────────────────────────────────────────────
    # PLOT 1: F1-Score Comparison Bar Chart
    # ─────────────────────────────────────────────────────────────────────────

    plt.figure(figsize=(14, 8))
    sorted_df = RESULTS_DF.sort_values('test_f1', ascending=True)

    colors = ['#FF6B6B' if 'Attention' not in name else '#4ECDC4'
              for name in sorted_df['experiment_name']]

    bars = plt.barh(sorted_df['experiment_name'], sorted_df['test_f1'], color=colors)

    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, sorted_df['test_f1'])):
        plt.text(val + 0.005, i, f'{val:.3f}', va='center', fontsize=9, fontweight='bold')

    plt.xlabel('Test F1-Score', fontsize=12, fontweight='bold')
    plt.ylabel('Experiment Name', fontsize=12, fontweight='bold')
    plt.title('Speech Emotion Recognition: All Experiments F1-Score Comparison',
              fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig('experiment_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    # ─────────────────────────────────────────────────────────────────────────
    # PLOT 2: Best Model Confusion Matrix
    # ─────────────────────────────────────────────────────────────────────────

    if 'confusion_matrix' in best_row and best_row['confusion_matrix'] is not None:
        try:
            # Handle string representation of list
            if isinstance(best_row['confusion_matrix'], str):
                import ast
                cm = np.array(ast.literal_eval(best_row['confusion_matrix']))
            else:
                cm = np.array(best_row['confusion_matrix'])

            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd',
                       xticklabels=EMOTION_LABELS, yticklabels=EMOTION_LABELS,
                       cbar_kws={'label': 'Count'}, linewidths=0.5, linecolor='gray')
            plt.title(f'Confusion Matrix: {best_row["experiment_name"]}\n'
                     f'Test F1: {best_row["test_f1"]:.4f}',
                     fontsize=14, fontweight='bold', pad=20)
            plt.ylabel('True Emotion', fontsize=12, fontweight='bold')
            plt.xlabel('Predicted Emotion', fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.savefig('best_model_confusion_matrix.png', dpi=300, bbox_inches='tight')
            plt.show()
        except Exception as e:
            print(f"⚠️  Could not plot confusion matrix: {e}")

    # ─────────────────────────────────────────────────────────────────────────
    # PLOT 3: Model Type Performance Comparison
    # ─────────────────────────────────────────────────────────────────────────

    plt.figure(figsize=(12, 6))
    model_type_performance = RESULTS_DF.groupby('model_type')['test_f1'].mean().sort_values()

    colors_model = plt.cm.viridis(np.linspace(0, 1, len(model_type_performance)))
    bars = plt.barh(model_type_performance.index, model_type_performance.values, color=colors_model)

    for i, (bar, val) in enumerate(zip(bars, model_type_performance.values)):
        plt.text(val + 0.005, i, f'{val:.3f}', va='center', fontsize=10, fontweight='bold')

    plt.xlabel('Average Test F1-Score', fontsize=12, fontweight='bold')
    plt.ylabel('Model Architecture', fontsize=12, fontweight='bold')
    plt.title('Performance by Model Architecture Type', fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig('model_type_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("\n✅ All visualizations saved successfully!")
    print("   - experiment_comparison.png")
    print("   - best_model_confusion_matrix.png")
    print("   - model_type_comparison.png")

print("\n" + "="*80)
print("🎉 COMPLETE! All experiments finished and results saved.")
print("="*80)

MessageError: Error: credential propagation was unsuccessful